In [1]:
import cobra
import os 
import sys
import numpy as np
import pandas as pd
import time
from sklearn.metrics import mean_squared_error
import scipy.stats as ss
from multiprocessing import Process,cpu_count,Manager
import pickle

In [2]:
ecpy_path = '../../../ecpy/'
sys.path.append(os.path.abspath(ecpy_path))
import utils
import ecpy

In [3]:
import importlib
importlib.reload(utils)
importlib.reload(ecpy)

<module 'ecpy' from '/Users/gangl/Documents/GitHub/Halo-GEM/ecpy/ecpy.py'>

In [4]:
def get_r_max(model):
    model.objective = 'Biomass_v1'
    model.objective_direction = 'max'
    return model.optimize()

In [5]:
def test_model(modelfile,      # json format
               kcats,          # dict() with enzyme kcats, in the unit of 1/h
               dfmws,          # dataframe with all protein weights, in the unit of kDa
               dfomics,        # contains absolute proteomics data
               dftot,          # contains the total protein abandance, in the unit of gram protein/gram CDW
               dfpheno,        # contains the P3HB specific synthetic rate
               condition_id,   # condition id
               sigma=0.5,
               huge_kcat=False # use huge kcat to release the kcat constraint
                ):
    
    print('Condition:',condition_id)
    model = cobra.io.load_json_model(modelfile)
    irrModel = ecpy.convertToIrrev(model)
    if huge_kcat: 
        for k,v in kcats.items(): kcats[k] = 1e8
            
    eModel = ecpy.convertToEnzymeModel(irrModel,kcats)
    
    # with proteomics
    measured, non_measured, prot_pool, enz_frac = ecpy.prepare_omics_for_one_condition(dfomics,
                                                                             dftot,
                                                                             dfmws,
                                                                             condition_id,
                                                                             eModel.enzymes)
    MWs = {ind:dfmws.loc[ind,'MW'] for ind in dfmws.index}
    
    with eModel:
        ecModel = ecpy.constrainPool(eModel,MWs, measured, non_measured,prot_pool*sigma)

        PHA = dfpheno.loc[condition_id,'P3HB']
        if ~np.isnan(PHA): 
            ecModel.reactions.PHA_secretion.lower_bound = PHA

        s1 = get_r_max(ecModel)
        r1 = s1.objective_value if s1.status == 'feasible' else -1
        print('Case 1: with omics constraints')
        print('  Status        :',s1.status)
        print('  growth_rate   :',r1)
        print('  glucose uptake:',s1.fluxes['Exchange_Glucopyranose'])
        print('  PHA           :',s1.fluxes['PHA_secretion'])
        print('  Protein pool  :',ecModel.reactions.get_by_id('prot_pool_exchange').upper_bound)
        print()

        
    # without proteomics
    with eModel:
        ecModel = ecpy.constrainPool(eModel,MWs, {}, eModel.enzymes,
                                     dftot.loc[condition_id,'Ptot']*enz_frac*sigma)
        if ~np.isnan(PHA): 
            ecModel.reactions.PHA_secretion.lower_bound = PHA

        s2 = get_r_max(ecModel)
        r2 = s2.objective_value if s2.status == 'feasible' else -1
        print('Case 2: without omics constraints')
        print('  Status        :',s2.status)
        print('  growth_rate   :',r2)
        print('  glucose uptake:',s2.fluxes['Exchange_Glucopyranose'])
        print('  PHA           :',s2.fluxes['PHA_secretion'])
        print('  Protein pool  :',ecModel.reactions.get_by_id('prot_pool_exchange').upper_bound)

    print(''.join(['\n']*2))
    return [r1,r2]

In [6]:
def simulate_a_single_kcat_set_on_all_datasets(datasets,kcats):
    start_time = time.time()
    results = {}
    for condition_id in datasets['dfpheno'].index:
        datasets['condition_id'] = condition_id
        res = test_model(kcats=kcats,**datasets)
        results[condition_id] = res
    print('Time:',time.time() - start_time)
    return results

In [7]:
class RV:
    def __init__(self,loc,scale):
        '''
        Normal distribution assumed
        loc, scale: the same as used in scipy.stats
        
        Use truncated normal distribution for kcat to ensure that all kcats are in (5.83e-10, 7900000.0) 1/s
        '''
       
        
        self.loc = loc
        self.scale = scale
        
        myclip_a = np.log10(1e-11 * 3600)
        my_mean = loc
        my_std = scale
        
        myclip_b = np.log10(1e7*3600)
        
        a, b = (myclip_a - my_mean) / my_std, (myclip_b - my_mean) / my_std
        

        self.rvf = ss.truncnorm(a,b,loc=my_mean,scale=my_std)
        
    def sample(self):
        '''
        Generate a random sample from the given prior distribution
        '''
        return self.rvf.rvs()

In [26]:
class smc_abc:
    def __init__(self,datasets,priors,epsilon,outfile):
        self.datasets   = datasets
        self.population = []
        self.epsilons   = [np.inf] # store the distance after each generation
        self.prior      = priors
        self.posterior  = priors # a dictionary with rxn_id as key, RV as value
        self.cores      = cpu_count()
        
        self.outfile    = outfile
        self.population = []  # a list of populations [p1,p2...]
        self.distances  = []    # a list of distances for particles in population
        self.simulations = 0  # number of simulations performed 
        
        self.population_size = 100
        self.generation_size = 128
        
        self.all_simulated_particle = []
        self.all_simulated_data     = []
        self.all_simulated_distance = []
        self.epsilon = epsilon
        
    def simulate_one(self,particle,index,Q):
        '''
        particle:  parameters 
        Q:      a multiprocessing.Queue object
        index:  the index in particles list
        '''
        
        res = simulate_a_single_kcat_set_on_all_datasets(self.datasets,particle)

        Q.put((index,res))
    
    def distance(self,res):
        y_sim = []
        y_exp = []
        for condition_id,lst in res.items():
            y_sim.extend(lst)
            y_exp.extend([self.datasets['dfpheno'].loc[condition_id,'SpecificGrowthRate']]*2)
            
        return mean_squared_error(y_exp,y_sim)
        
    def calculate_distances_parallel(self,particles):
        Q = Manager().Queue()
        jobs = [Process(target=self.simulate_one,args=(particle,index,Q)) 
                               for index,particle in enumerate(particles)]
        
        for p in jobs: p.start()
        for p in jobs: p.join()
        
        distances = [None for _ in range(len(particles))]
        simulated_data = [None for _ in range(len(particles))]

        for index,res in [Q.get(timeout=1) for p in jobs]: 
            distances[index] = self.distance(res)
            simulated_data[index] = res
        
        # save all simulated results
        self.all_simulated_data.extend(simulated_data)
        self.all_simulated_distance.extend(distances)
        self.all_simulated_particle.extend(particles)
        
        return distances,simulated_data
        
    def simulate_a_generation(self):
        particles_t, simulated_data_t, distances_t = [], [], []
        while len(particles_t) < self.generation_size:
            self.simulations += self.cores
            particles = [{idp: 10**rv.sample() for idp,rv in self.posterior.items()} for i in range(self.cores)]

            distances,simulated_data = self.calculate_distances_parallel(particles)
            
            particles_t.extend(particles)
            simulated_data_t.extend(simulated_data)
            distances_t.extend(distances)
        
        return particles_t, simulated_data_t, distances_t
    
    def update_population(self,particles_t, simulated_data_t, distances_t):
        print ('updating population')
        # save first generation
        if len(self.population) == 0:
            self.population_t0 = particles_t
            self.distances_t0 = distances_t
            self.simulated_data_t0 = simulated_data_t
        
        
        combined_particles = np.array(self.population + particles_t)
        combined_distances = np.array(self.distances + distances_t)
        combined_simulated = np.array(self.simulated_data + simulated_data_t)
        
        sort_index = np.argsort(combined_distances)
        self.population = list(combined_particles[sort_index][:self.population_size])
        self.distances = list(combined_distances[sort_index][:self.population_size])
        self.simulated_data = list(combined_simulated[sort_index][:self.population_size])
        self.epsilons.append(np.max(self.distances))
        
        print('Model: epsilon=',str(self.epsilons[-1]))
        
    def update_posterior(self):
        print ('Updating prior')
        parameters = dict()   # {'Protein_Tm':[]}
        for particle in self.population:
            for p,val in particle.items(): 
                lst = parameters.get(p,[])
                lst.append(np.log10(val))
                parameters[p] =lst
        
        for p, lst in parameters.items():
            self.posterior[p] = RV(loc = np.mean(lst), scale = np.std(lst))
        
        
    def run_simulation(self):
        while self.epsilons[-1] > self.epsilon:
            particles_t, simulated_data_t, distances_t = self.simulate_a_generation()
            self.update_population(particles_t, simulated_data_t, distances_t)
            self.update_posterior()
            pickle.dump(self,open(self.outfile,'wb'))
        

In [18]:
def build_priors(kcats,df_enz_kcat):
    # in prior, kcat is in log10-transformed, 1/h
    # in kcats, only keys will be used
    # in df_enz_kcat, value are in the unit of 1/s, log1o-transformed
    
    priors = dict()
    for (enz_id,rxn_id) in kcats.keys():
        loc   = np.log10(10**df_enz_kcat.loc[rxn_id,'log10_kcat_mean']*3600)
        scale = np.log10(10**df_enz_kcat.loc[rxn_id,'log10_kcat_std']*3600)
        priors[(enz_id,rxn_id)] = RV(loc,scale)
    return priors

In [19]:
modelfile = '../../../ModelFiles/json/Halo_GEM_v1.json'

In [20]:
dfomics = pd.read_csv('../proteomics/protein_abandance_mean.csv',index_col=0)
dftot = pd.read_csv('../proteomics/total_protein_abandance_mean.csv',index_col=0)
dfmws = pd.read_csv('../Results/protein_mws.csv',index_col=0)
dfmws = dfmws/1000
dfpheno = pd.read_csv('../proteomics/phynotype.csv',index_col=0,comment='#')
df_enz_kcat = pd.read_csv('../Results/mapped_kcats.csv',index_col=0)

In [21]:
model = cobra.io.load_json_model(modelfile)
irrModel = ecpy.convertToIrrev(model)
kcats = ecpy.prepare_kcats_dict(irrModel,df_enz_kcat,'log10_kcat_mean')
del model,irrModel

In [22]:
datasets = {
    'modelfile': modelfile,
    'dfomics'   : dfomics,
    'dfmws'     : dfmws,
    'dfpheno'     : dfpheno,
    'dftot'       : dftot,
}

In [23]:
priors = build_priors(kcats,df_enz_kcat)

In [27]:
epsilon = 0
outfile = '../Results/smc_abc.pkl'
experiment = smc_abc(datasets,priors,epsilon,outfile)

In [ ]:
experiment.run_simulation()

results = simulate_a_single_kcat_set_on_all_datasets(datasets,kcats)

for condition_id in dfomics.columns:
    if condition_id != 'NACL60': continue
    test_model(modelfile,      # json format
               df_enz_kcat,    # dataframe with enzyme kcats, in the unit of 1/s, log10transformed
               'log10_kcat_mean',   # column name for kcat
               dfmws,          # dataframe with all protein weights, in the unit of kDa
               dfomics,        # contains absolute proteomics data
               dftot,          # contains the total protein abandance, in the unit of gram protein/gram cell dry mass
               dfpheno,        # contains the P3HB specific synthetic rate
               condition_id,   # condition id
               sigma=0.5,
               huge_kcat=False )
    print()

start_time = time.time()
for condition_id in dfomics.columns:
    print(condition_id)
    if condition_id != 'NACL60': continue
    test_model(modelfile,      # json format
               df_enz_kcat,    # dataframe with enzyme kcats, in the unit of 1/s, log10transformed
               'log10_kcat_mean',   # column name for kcat
               dfmws,          # dataframe with all protein weights, in the unit of kDa
               dfomics,        # contains absolute proteomics data
               dftot,          # contains the total protein abandance, in the unit of gram protein/gram cell dry mass
               dfpheno,        # contains the P3HB specific synthetic rate
               condition_id,   # condition id
               sigma=0.5,
               huge_kcat=True )
    print()
print('Time:',time.time() - start_time)